In [ ]:
jiant_url = "https://drive.google.com/u/0/uc?id=1lFbvdomHhLNSxm37OBxGYsinWy3vYC9U&export=download"
!wget "$jiant_url" -O jiant.zip
!unzip jiant.zip -d ./
!rm jiant.zip

%cd jiant
!pip install -r requirements-no-torch.txt
!pip install --no-deps -e ./

Restart runtime after installing libs

In [2]:
%cd /content/jiant

import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

def init_task_config(task_name):
  jiant_task = task_name
  if(task_name == "sup-squad" or task_name == "sup-babi"):
    jiant_task = "coref"  # use coref task to probe supporting facts task because of the analog structure of jiant EP json format

  os.makedirs("/content/tasks/configs/", exist_ok=True)
  os.makedirs(f"/content/tasks/data/{task_name}", exist_ok=True)

  py_io.write_json({
    "task": jiant_task,
    "paths": {
      "train": f"/content/tasks/data/{task_name}/train.jsonl",
      "val":   f"/content/tasks/data/{task_name}/val.jsonl",
    },
    "name": jiant_task
  }, f"/content/tasks/configs/{task_name}_config.json")

/content/jiant


In [3]:
task_names = [
              #"ner", 
              #"semeval", 
              #"coref",  # supported by jiant 
              #"srl",                    # additional task for other badge     
              "sup-squad", 
              #"ques"
             ]  

for task_name in task_names:
  init_task_config(task_name)

In [4]:
# contains data of all probing tasks above
task_data_url = "https://drive.google.com/u/0/uc?id=1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f&export=download"
!wget "$task_data_url" -O /content/task_data.zip
!unzip /content/task_data.zip -d /content/tasks/
!rm /content/task_data.zip

--2021-06-27 21:55:24--  https://drive.google.com/u/0/uc?id=1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f&export=download
Resolving drive.google.com (drive.google.com)... 172.217.9.206, 2607:f8b0:4004:807::200e
Connecting to drive.google.com (drive.google.com)|172.217.9.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9v2civmkn181540abn2dmfjoc1gbits6/1624830900000/13675844253040106205/*/1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f?e=download [following]
--2021-06-27 21:55:25--  https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9v2civmkn181540abn2dmfjoc1gbits6/1624830900000/13675844253040106205/*/1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f?e=download
Resolving doc-04-18-docs.googleusercontent.com (doc-04-18-docs.googleusercontent.com)... 142.250.73.225, 2607:f8b0:4004:82a::2001
Connecting to doc-04-18-docs.googleusercontent.com (doc-04-18-doc

In [5]:
import jiant.proj.main.export_model as export_model

models = [
          #"bert-base-uncased", 
          "csarron/bert-base-uncased-squad-v1"
          ]

for model in models:
  export_model.export_model(
      hf_pretrained_model_name_or_path=model,
      output_base_path=f"/content/models/{model}",
  )

Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForPreTraining: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at csarron/bert-base-uncased-squad-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.

In [6]:
import jiant.shared.caching as caching
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache

# Tokenize and cache each task
def tokenize(task_names, models):
  for task_name in task_names:
    for model in models:
      tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
          task_config_path=f"/content/tasks/configs/{task_name}_config.json",
          hf_pretrained_model_name_or_path=model,
          output_dir=f"/content/cache/{task_name}",
          phases=["train", "val"],
      ))

tokenize(task_names, models)

CorefTask
  [train]: /content/tasks/data/sup-squad/train.jsonl
  [val]: /content/tasks/data/sup-squad/val.jsonl


In [7]:
row = caching.ChunkedFilesDataCache(f"/content/cache/{task_names[0]}/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)
print(row.spans)
print(row.tokens[row.spans[0][0]: row.spans[0][1]+1])
#print(row.tokens[row.spans[1][0]: row.spans[1][1]+1])

[  101  2054  2079 26092  7486  2903  2003  1996  3114  2005  1996 17702
  1997  2824  1998  3494  1999  1996  6331  1998 21288  1029  2429  2000
  7842  4048  2232  2632  1011 20934 22510  2072  1010  1996 21288  2001
 24843  2426 24485 10586  1998  8956  2015  1010  1998  6936  2011  8135
  1998  5181  1010  2077  2009  2001 16367  1012  2049  2653  2001  2714
  2000  1996 24109  2653  1012  1031 11091  2734  1033  1996 21288 23412
  3441  1997  2116  1997  1996  2111  1998  2824 22906  1999  3644  1998
  3017  6730  2808  1006  9092 27573  1010  6331  1007  1998 13347  2389
  3906  1006  9706 10085  2854 21890  1010  3054  8180  2232  1007  1010
  2348  2009 12980  1999  2116  4751  1012  4205  1010 28178  1010  7240
  1010  1041  5677  1010  2016 14431  1010  8181  1010  2843  1010  2003
 22444  2884  1010  7527  1010  6213  1010  3312  1010  3105  1010  6892
  8093  2080  1010  2585  1010  9168  1010 14063  1010 12005  7377  1010
 15617  1010  7158  1010  9952  1010 27838  7507  4

In [12]:
import jiant.proj.main.scripts.configurator as configurator

def create_jiant_task_config(task_name):
  jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
      task_config_base_path="/content/tasks/configs",
      task_cache_base_path="/content/cache",
      train_task_name_list=[task_name],
      val_task_name_list=[task_name],
      train_batch_size=8,
      eval_batch_size=2,
      epochs=5,
      num_gpus=1,
  ).create_config()
  os.makedirs("/content/run_configs/", exist_ok=True)
  py_io.write_json(jiant_run_config, f"/content/run_configs/{task_name}_run_config.json")
  #display.show_json(jiant_run_config)

In [13]:
# You may restart runtime when you modify the jiant source code

import jiant.proj.main.runscript as main_runscript

def run_probing_task(task_name, model_name="bert-base-uncased", num_layers=1):
  run_args = main_runscript.RunConfiguration(
      jiant_task_container_config_path=f"/content/run_configs/{task_name}_run_config.json",
      output_dir=f"/content/runs/{task_name}",
      hf_pretrained_model_name_or_path=model_name,
      model_path=f"/content/models/{model_name}/model/model.p",
      model_config_path=f"/content/models/{model_name}/model/config.json",
      learning_rate=1e-3,
      eval_every_steps=1000,
      do_train=True,
      do_val=True,
      do_save=True,
      force_overwrite=True,
      num_hidden_layers=num_layers,
  )
  return main_runscript.run_loop(run_args)

In [ ]:
# test probing

num_layers = list(range(1, 13))      # from 1 to 12 layers

result = "model\ttask\tlayer\tacc\tf1_macro\n"

for model in models:
  for task in task_names:
    for n_layers in num_layers:
      create_jiant_task_config(task)
      probing_output = run_probing_task(task, model, n_layers)
      acc = str(probing_output[task]["metrics"]["minor"]["acc"])
      f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
      result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"

In [11]:
print(result)

model	task	layer	acc	f1_macro
csarron/bert-base-uncased-squad-v1	sup-squad	1	0.75	0.4166666666666667
csarron/bert-base-uncased-squad-v1	sup-squad	2	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	3	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	4	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	5	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	6	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	7	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	8	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	9	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	10	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	11	0.75	0.42857142857142855
csarron/bert-base-uncased-squad-v1	sup-squad	12	0.75	0.42857142857142855



In [ ]:
def plot_task(...):
  ...

In [11]:
# Probe NER task with bert-base and bert-squad and plot macro f1 score

In [ ]:
# Probe SEMEVAL task with bert-base and bert-squad and plot macro f1 score

In [ ]:
# Probe COREF task with bert-base and bert-squad and plot macro f1 score

In [ ]:
# Probe SUP-SQUAD task with bert-base and bert-squad and plot macro f1 score

In [ ]:
# Probe QUES task with bert-base and bert-squad and plot macro f1 score to compare the models